In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "four_square.csv"
four_square_df = pd.read_csv(file_path)

# Remove the specified columns
columns_to_remove = ['venue_id', 'venue_latitude', 'venue_longitude']
four_square_df.drop(columns=columns_to_remove, inplace=True)

# Save the modified DataFrame to a new CSV file
modified_file_path = "four_square_modified.csv"
four_square_df.to_csv(modified_file_path, index=False)

# Display the first few rows of the modified DataFrame to confirm
four_square_df.head()


,station_name,latitude,longitude,venue_name,venue_category,venue_address,venue_distance
0,10th & Cambie,49.262487,-123.114397,Winona Park,Landmarks and Outdoors,400 W 59th Ave,105
1,10th & Cambie,49.262487,-123.114397,Whole Foods,Retail,510 8th Ave W,173
2,10th & Cambie,49.262487,-123.114397,La Taqueria Pinche Taco Shop,Taco Restaurant,2450 Yukon St,183
3,10th & Cambie,49.262487,-123.114397,Elysian Coffee Broadway,Coffee Shop,590 Broadway W,216
4,10th & Cambie,49.262487,-123.114397,Cactus Club Cafe Broadway + Ash,American Restaurant,575 West Broadway,226


In [6]:
# Load the CSV file into a DataFrame
file_path = "four_square.csv"
four_square_df = pd.read_csv(file_path)

# Remove the specified columns
columns_to_remove = ['venue_id', 'venue_latitude', 'venue_longitude']
four_square_df.drop(columns=columns_to_remove, inplace=True)

# Rename 'venue_name' to 'business_name' and 'venue_category' to 'categories'
four_square_df.rename(columns={'venue_name': 'business_name', 'venue_category': 'categories'}, inplace=True)

# Add new columns 'review_count', 'rating', 'price', and 'phone' with None values
four_square_df.insert(four_square_df.columns.get_loc('business_name') + 1, 'review_count', None)
four_square_df.insert(four_square_df.columns.get_loc('review_count') + 1, 'rating', None)
four_square_df.insert(four_square_df.columns.get_loc('categories') + 1, 'price', None)
four_square_df.insert(four_square_df.columns.get_loc('price') + 1, 'phone', None)

# Move 'venue_distance' next to 'phone' and rename it to 'distance'
distance_column = four_square_df.pop('venue_distance')
four_square_df.insert(four_square_df.columns.get_loc('phone') + 1, 'distance', distance_column)

# Rename 'venue_address' to 'address' and move it to the end
address_column = four_square_df.pop('venue_address')
four_square_df['address'] = address_column

# Save the modified DataFrame to a new CSV file
modified_file_path = "four_square_modified.csv"
four_square_df.to_csv(modified_file_path, index=False)


# Display the first few rows of the modified DataFrame to confirm
four_square_df.head()


,station_name,latitude,longitude,business_name,review_count,rating,categories,price,phone,distance,address
0,10th & Cambie,49.262487,-123.114397,Winona Park,None,None,Landmarks and Outdoors,None,None,105,400 W 59th Ave
1,10th & Cambie,49.262487,-123.114397,Whole Foods,None,None,Retail,None,None,173,510 8th Ave W
2,10th & Cambie,49.262487,-123.114397,La Taqueria Pinche Taco Shop,None,None,Taco Restaurant,None,None,183,2450 Yukon St
3,10th & Cambie,49.262487,-123.114397,Elysian Coffee Broadway,None,None,Coffee Shop,None,None,216,590 Broadway W
4,10th & Cambie,49.262487,-123.114397,Cactus Club Cafe Broadway + Ash,None,None,American Restaurant,None,None,226,575 West Broadway


In [9]:


# Load the two CSV files into DataFrames
foursquare_path = "four_square_modified.csv"  # Replace with your actual file path
yelp_path = "yelp_businesses_near_bike_stations.csv"  # Replace with your actual file path

foursquare_df = pd.read_csv(foursquare_path)
yelp_df = pd.read_csv(yelp_path)

# Add the 'source' column to each DataFrame
foursquare_df['source'] = 'FourSquare'
yelp_df['source'] = 'Yelp'

# Merge/union the two DataFrames
merged_df = pd.concat([foursquare_df, yelp_df], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_file_path = "merged_restaurants.csv"  # Replace with your desired file path
merged_df.to_csv(merged_file_path, index=False)



# Display the first few rows of the merged DataFrame to confirm
merged_df.head()


,station_name,latitude,longitude,business_name,review_count,rating,categories,price,phone,distance,address,source
0,10th & Cambie,49.262487,-123.114397,Winona Park,NaN,NaN,Landmarks and Outdoors,NaN,NaN,105.0,400 W 59th Ave,FourSquare
1,10th & Cambie,49.262487,-123.114397,Whole Foods,NaN,NaN,Retail,NaN,NaN,173.0,510 8th Ave W,FourSquare
2,10th & Cambie,49.262487,-123.114397,La Taqueria Pinche Taco Shop,NaN,NaN,Taco Restaurant,NaN,NaN,183.0,2450 Yukon St,FourSquare
3,10th & Cambie,49.262487,-123.114397,Elysian Coffee Broadway,NaN,NaN,Coffee Shop,NaN,NaN,216.0,590 Broadway W,FourSquare
4,10th & Cambie,49.262487,-123.114397,Cactus Club Cafe Broadway + Ash,NaN,NaN,American Restaurant,NaN,NaN,226.0,575 West Broadway,FourSquare


In [14]:
#Join the data from Part 1 with the data from Part 2 to create a new dataframe.
# Load the CSV file into a DataFrame
bike_stations_path = "Bike_stations.csv"  # Replace with your actual file path
bike_stations_df = pd.read_csv(bike_stations_path)



# Rename 'name to venue_name'
bike_stations_df.rename(columns={'name': 'station_name'}, inplace=True)

# Display the first few rows of the DataFrame to confirm
bike_stations_df.head()

,station_name,latitude,longitude,free_bikes,ebikes,empty_slots
0,10th & Cambie,49.262487,-123.114397,1,1,33
1,Yaletown-Roundhouse Station,49.274566,-123.121817,14,0,0
2,Dunsmuir & Beatty,49.279764,-123.110154,1,1,24
3,12th & Yukon (City Hall),49.260599,-123.113504,1,0,13
4,8th & Ash,49.264215,-123.117772,2,0,14


In [ ]:
rest_bikes_df = pd.merge(bike_stations_df, merged_df, on=['latitude', 'longitude'], how='inner')

# Merge the resulting DataFrame with Foursquare data on latitude and longitude
merged_df = pd.merge(merged_df_yelp, foursquare_df, on=['latitude', 'longitude'], how='inner')

# Print the head of the merged dataframe
print(merged_df.head())

In [ ]:
#Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship 
#you discoved through this visualization.

In [ ]:
#Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - 
#make sure to create your database in your project's data/ directory!)


In [ ]:
#Look at the data before and after the join to validate your data.